In [1]:
import pandas as pd
import numpy as np
from psmpy import PsmPy
from psmpy.functions import cohenD
from psmpy.plotting import *

In [3]:
treatmentCol = 'COVID'
OutcomeCol = 'Dyspnea'
excludedColumns = []
df = pd.read_csv("C:/Users/17172/Desktop/iRCT/datasets/COVID3_4Nodes3.dat")
df.index = range(1, len(df)+1, 1)
df = df.replace(to_replace='No', value=0)
df = df.replace(to_replace='Yes', value=1)
df = df.replace(to_replace='Negtive', value=0)
df = df.replace(to_replace='Positive', value=1)
df

,ED_Visit,Dyspnea,COPD,COVID
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,1
5,0,0,0,0
...,...,...,...,...
49996,0,1,0,1
49997,0,0,0,0
49998,0,0,0,0
49999,0,0,0,0


In [6]:
psm = PsmPy(df, treatment=treatmentCol, indx=df.index, exclude=[])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [38]:
smf.ols("Dyspnea ~ COVID", data=df).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.0047,0.001,8.475,0.000,0.004,0.006
COVID,0.1022,0.002,58.472,0.000,0.099,0.106


In [39]:
from sklearn.linear_model import LogisticRegression

T = treatmentCol
Y = OutcomeCol
X = df.columns.drop([T, Y])

ps_model = LogisticRegression(C=1e6).fit(df[X], df[T])
data_ps = df.assign(propensity_score=ps_model.predict_proba(df[X])[:, 1])

In [40]:
weight_t = 1/data_ps.query("COVID==1")["propensity_score"]
weight_nt = 1/(1-data_ps.query("COVID==0")["propensity_score"])
print("Original Sample Size", df.shape[0])
print("Treated Population Sample Size", sum(weight_t))
print("Untreated Population Sample Size", sum(weight_nt))

Original Sample Size 50000
Treated Population Sample Size 50000.431836647294
Untreated Population Sample Size 49999.98396816923


In [41]:
weight = ((data_ps[treatmentCol]-data_ps["propensity_score"]) /
          (data_ps["propensity_score"]*(1-data_ps["propensity_score"])))


y1 = sum(data_ps.query("COVID==1")[OutcomeCol]*weight_t) / len(df)
y0 = sum(data_ps.query("COVID==0")[OutcomeCol]*weight_nt) / len(df)

ate = np.mean(weight * data_ps[OutcomeCol])
print("ATE:", ate)

ATE: 0.08994794374830722
